In [1]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
from keras.models import Model
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, Input, AveragePooling2D, Lambda
from urllib import request
from keras.callbacks import TensorBoard
import tempfile
from keras import optimizers
from keras.utils import Sequence
import numpy as np
from tqdm import tqdm

/home/trule/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## Define 
root_dir = '../data/kaggle/dog-breed/'
labels_path = os.path.join(root_dir, 'labels.csv')
train_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'test')

In [3]:
def fetch_data():
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)
        zip_path = os.path.join(tempfile.gettempdir(), 'dog-breed.zip')
        print('Start download!')
        request.urlretrieve('https://kienle.blob.core.windows.net/public/kaggle/dog-breed.zip', zip_path)
        print('Start unzip')
        import zipfile
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(root_dir)
        with zipfile.ZipFile(os.path.join(root_dir,'test.zip'), 'r') as zip_ref:
            zip_ref.extractall(root_dir)
        with zipfile.ZipFile(os.path.join(root_dir,'train.zip'), 'r') as zip_ref:
            zip_ref.extractall(root_dir)
        os.remove(zip_path)
        os.remove(os.path.join(root_dir,'train.zip'))
        os.remove(os.path.join(root_dir,'test.zip'))
        print('Done')
fetch_data()

Start download!
Start unzip
Done


In [4]:
# Confirm all data are ready 
# Expected output: labels.csv  sample_submission.csv  test  train
!ls $root_dir

labels.csv  sample_submission.csv  test  train


In [5]:
pd_images = pd.read_csv(labels_path)

In [6]:
pd_images.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [7]:
train_valid_generator = ImageDataGenerator().flow_from_dataframe(pd_images, train_dir, batch_size=32, x_col='id', y_col='breed', has_ext=False, target_size=(224,224))

Found 10222 images belonging to 120 classes.


In [8]:
num_classes = len(train_valid_generator.class_indices)

In [9]:
from keras.applications import vgg16
class FeaturesExtractor():
    def __init__(self, input_shape):
        self.input_shape = input_shape
        model = vgg16.VGG16(include_top=False, input_shape=self.input_shape, weights='imagenet')
        inputs = Input(self.input_shape)
        x = inputs
        x = Lambda(vgg16.preprocess_input, name='preprocessing')(x)
        x = model(x)
        x = GlobalAveragePooling2D()(x)
        model = Model(inputs, x)
        self.model = model
    def predict(self, X):
        return self.model.predict(X)

In [10]:
vgg_features_extractor = FeaturesExtractor((224,224,3))

In [11]:
def create_simple_model(num_classes, input_shape):
    inputs = Input(input_shape)
    x = inputs
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, x)


In [15]:
X_train = np.zeros((train_valid_generator.n, 512))
y_train = np.zeros((train_valid_generator.n, 120))
for i in tqdm(range(len(train_valid_generator)), ncols=100):
    temp_x, temp_y = train_valid_generator[i]
    start_index = i*train_valid_generator.batch_size
    X_train[start_index:start_index + temp_x.shape[0]] = vgg_features_extractor.predict(temp_x)
    y_train[start_index:start_index + temp_x.shape[0]] = temp_y

100%|█████████████████████████████████████████████████████████████| 320/320 [01:04<00:00,  3.56it/s]


In [16]:
model = create_simple_model(num_classes, (512,))

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               61560     
Total params: 61,560
Trainable params: 61,560
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(X_train, y_train, batch_size=256, epochs=100, callbacks=[TensorBoard('./data/logs/run7/')], validation_split=0.1)

Train on 9199 samples, validate on 1023 samples
Epoch 1/100
9199/9199 [==============================] - 0s 37us/step - loss: 14.8212 - acc: 0.0171 - val_loss: 12.5191 - val_acc: 0.0684
Epoch 2/100
9199/9199 [==============================] - 0s 10us/step - loss: 13.6392 - acc: 0.0528 - val_loss: 10.8400 - val_acc: 0.1378
Epoch 3/100
9199/9199 [==============================] - 0s 10us/step - loss: 12.4386 - acc: 0.0937 - val_loss: 9.2404 - val_acc: 0.1965
Epoch 4/100
9199/9199 [==============================] - 0s 10us/step - loss: 11.1605 - acc: 0.1435 - val_loss: 8.1093 - val_acc: 0.2796
Epoch 5/100
9199/9199 [==============================] - 0s 10us/step - loss: 10.1119 - acc: 0.1984 - val_loss: 7.3838 - val_acc: 0.3382
Epoch 6/100
9199/9199 [==============================] - 0s 10us/step - loss: 9.1445 - acc: 0.2434 - val_loss: 6.5510 - val_acc: 0.3783
Epoch 7/100
9199/9199 [==============================] - 0s 10us/step - loss: 8.3079 - acc: 0.2817 - val_loss: 5.9451 - val_acc: 

9199/9199 [==============================] - 0s 10us/step - loss: 2.6688 - acc: 0.6535 - val_loss: 3.1584 - val_acc: 0.5982
Epoch 61/100
9199/9199 [==============================] - 0s 10us/step - loss: 2.6688 - acc: 0.6567 - val_loss: 3.0849 - val_acc: 0.6139
Epoch 62/100
9199/9199 [==============================] - 0s 10us/step - loss: 2.6336 - acc: 0.6580 - val_loss: 3.1064 - val_acc: 0.6031
Epoch 63/100
9199/9199 [==============================] - 0s 10us/step - loss: 2.7029 - acc: 0.6520 - val_loss: 3.1347 - val_acc: 0.6041
Epoch 64/100
9199/9199 [==============================] - 0s 10us/step - loss: 2.6709 - acc: 0.6575 - val_loss: 3.1190 - val_acc: 0.6061
Epoch 65/100
9199/9199 [==============================] - 0s 10us/step - loss: 2.6404 - acc: 0.6572 - val_loss: 3.0690 - val_acc: 0.6051
Epoch 66/100
9199/9199 [==============================] - 0s 10us/step - loss: 2.5868 - acc: 0.6719 - val_loss: 3.0662 - val_acc: 0.6080
Epoch 67/100
9199/9199 [==============================